In [1]:
from __future__ import division, print_function, absolute_import
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import numpy
import PIL
from PIL import Image
np.random.seed(1337)  # for reproducibility

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import RMSprop
from keras import backend as K
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [74]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10

# Network Parameters
num_hidden_1 = 64 # 1st layer num features
num_hidden_2 = 16 # 2nd layer num features (the latent dim)
num_input = 43 
num_classes = 5

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
    'classifier_h': tf.Variable(tf.random_normal([num_hidden_2, num_classes])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([num_input])),
    'classifier_b': tf.Variable(tf.random_normal([num_classes])),
}

In [75]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op

label_pred = tf.nn.softmax(tf.add(tf.matmul(encoder_op, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [31]:
from numpy import genfromtxt
my_data = genfromtxt('/home/aniruddha/Documents/deep_learning_papers/DataBiplab/Cgamma_VHc.csv', delimiter=',')

In [32]:
data = my_data[1:,2:]
print(data.shape)

(291, 43)


In [33]:
print(data)

[[0.03615451 0.03379966 0.03095512 ... 0.04624293 0.05025645 0.00981916]
 [0.05635222 0.03707358 0.03283687 ... 0.02561918 0.02236751 0.00540298]
 [0.03499755 0.02029016 0.03339879 ... 0.08537867 0.03078398 0.00810179]
 ...
 [0.0391872  0.0566566  0.04518491 ... 0.04750742 0.04829812 0.01046296]
 [0.03182685 0.02146965 0.02652374 ... 0.03881095 0.04491708 0.03071279]
 [0.02908418 0.03307716 0.03758971 ... 0.04509031 0.04975136 0.04267977]]


In [34]:
print(data.max())

0.162946105003357


In [35]:
data = np.array(data)
print(data.shape)

(291, 43)


In [36]:
import pandas
colnames = ['Latitude', 'Longitude', 'CType', 'Remarks', 'Index']
data1 = pandas.read_csv('/home/aniruddha/Documents/deep_learning_papers/DataBiplab/CropClean.csv', names=colnames)

In [37]:
ctype = data1.CType.tolist()
print(ctype)

['CType', 'PD', 'PD', 'PD', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'JW', 'SC', 'SC', 'SC', 'PP', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'BA', 'BA', 'SC', 'SC', 'SC', 'JW', 'PD', 'SC', 'BA', 'SC', 'BA', 'PD', 'PD', 'SC', 'PP', 'PD', 'PD', 'PD', 'PD', 'PD', 'BA', 'SC', 'SC', 'BA', 'BA', 'JW', 'JW', 'SC', 'PD', 'BA', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'PD', 'BA', 'SC', 'PD', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'JW', 'PD', 'PD', 'SC', 'JW', 'PD', 'PD', 'BA', 'BA', 'SC', 'SC', 'SC', 'BA', 'PP', 'PP', 'BA', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'PP', 'SC', 'SC', 'SC', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'JW', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'BA', 'BA', 'PD', 'PD', 'SC', 'PD', 'SC', 'PD', 'BA', 'JW', 'PD', 'SC', 'JW', 'SC', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'PD', 'SC', 'SC', 'PD', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'JW', 'PD', 'SC', 'SC', 'JW', 

In [38]:
p = np.array(ctype)
print(p.shape)

(292,)


In [39]:
lbl1 = ctype
label = []
for i in range(1,292):
    if(lbl1[i] == "PD"):
        label += [0]
    if(lbl1[i] == "JW"):
        label += [1]
    if(lbl1[i] == "PP"):
        label += [2]
    if(lbl1[i] == "SC"):
        label += [3]
    if(lbl1[i] == "BA"):
        label += [4]
print(label)

[0, 0, 0, 3, 1, 3, 3, 3, 3, 1, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 1, 0, 3, 4, 3, 4, 0, 0, 3, 2, 0, 0, 0, 0, 0, 4, 3, 3, 4, 4, 1, 1, 3, 0, 4, 1, 3, 3, 3, 3, 3, 0, 0, 4, 3, 0, 0, 3, 3, 3, 3, 3, 1, 0, 0, 3, 1, 0, 0, 4, 4, 3, 3, 3, 4, 2, 2, 4, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3, 3, 3, 1, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 0, 0, 3, 0, 3, 0, 4, 1, 0, 3, 1, 3, 3, 1, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 3, 1, 3, 3, 3, 3, 3, 3, 0, 1, 0, 3, 3, 1, 3, 3, 3, 3, 3, 3, 0, 2, 3, 3, 3, 3, 0, 0, 0, 0, 1, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 4, 4, 3, 0, 4, 0, 3, 3, 3, 3, 0, 3, 0, 3, 2, 3, 0, 3, 1, 4, 3, 0, 0, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 4, 1, 0, 0, 2, 3, 3, 3, 0, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 0, 4, 4, 4, 0, 0, 0, 0, 3, 3, 3, 3, 4, 3, 3, 4, 4, 3, 3, 1, 3, 4, 1, 3, 3, 3, 3, 4, 0, 0]


In [40]:
label = np.array(label)
print(label.shape)

(291,)


In [65]:
# 80% train 20% test (NDVIclean)
X_train = data[0:232,:]
X_test = data[232:,:]
y_train = label[0:232]
y_test = label[232:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(232, 43)
(59, 43)
(232,)
(59,)


In [66]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(232, 5)
(59, 5)


In [67]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 17
num_batch = 13

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.71281793
Epoch: 2 cost = 0.63972603
Epoch: 3 cost = 0.58578687
Epoch: 4 cost = 0.54818192
Epoch: 5 cost = 0.52230811
Epoch: 6 cost = 0.50403340
Epoch: 7 cost = 0.49043912
Epoch: 8 cost = 0.47964846
Epoch: 9 cost = 0.47048919
Epoch: 10 cost = 0.46224207
Epoch: 11 cost = 0.45448549
Epoch: 12 cost = 0.44699916
Epoch: 13 cost = 0.43970046
Epoch: 14 cost = 0.43260068
Epoch: 15 cost = 0.42577611
Epoch: 16 cost = 0.41935053
Epoch: 17 cost = 0.41348004
Epoch: 18 cost = 0.40832877
Epoch: 19 cost = 0.40402627
Epoch: 20 cost = 0.40061808
Epoch: 21 cost = 0.39803815
Epoch: 22 cost = 0.39612717
Epoch: 23 cost = 0.39468657
Epoch: 24 cost = 0.39353264
Epoch: 25 cost = 0.39252485
Epoch: 26 cost = 0.39156877
Epoch: 27 cost = 0.39060690
Epoch: 28 cost = 0.38960758
Epoch: 29 cost = 0.38855613
Epoch: 30 cost = 0.38744866
Epoch: 31 cost = 0.38628791
Epoch: 32 cost = 0.38508080
Epoch: 33 cost = 0.38383643
Epoch: 34 cost = 0.38256414
Epoch: 35 cost = 0.38127191
Epoch: 36 cost = 0.37996488
E

Epoch: 393 cost = 0.01976972
Epoch: 394 cost = 0.01954253
Epoch: 395 cost = 0.01930146
Epoch: 396 cost = 0.01905696
Epoch: 397 cost = 0.01890166
Epoch: 398 cost = 0.02019196
Epoch: 399 cost = 0.03141152
Epoch: 400 cost = 0.02770684
Epoch: 401 cost = 0.01661841
Epoch: 402 cost = 0.01731216
Epoch: 403 cost = 0.01700940
Epoch: 404 cost = 0.01675976
Epoch: 405 cost = 0.01662960
Epoch: 406 cost = 0.01653934
Epoch: 407 cost = 0.01644625
Epoch: 408 cost = 0.01633679
Epoch: 409 cost = 0.01620897
Epoch: 410 cost = 0.01606407
Epoch: 411 cost = 0.01590465
Epoch: 412 cost = 0.01573366
Epoch: 413 cost = 0.01555391
Epoch: 414 cost = 0.01536620
Epoch: 415 cost = 0.01516768
Epoch: 416 cost = 0.01495819
Epoch: 417 cost = 0.01475581
Epoch: 418 cost = 0.01467145
Epoch: 419 cost = 0.01558203
Epoch: 420 cost = 0.02342106
Epoch: 421 cost = 0.02537094
Epoch: 422 cost = 0.01257953
Epoch: 423 cost = 0.01311955
Epoch: 424 cost = 0.01293204
Epoch: 425 cost = 0.01275928
Epoch: 426 cost = 0.01266720
Epoch: 427 cos

Epoch: 691 cost = 0.00008663
Epoch: 692 cost = 0.00009101
Epoch: 693 cost = 0.00009669
Epoch: 694 cost = 0.00010400
Epoch: 695 cost = 0.00011078
Epoch: 696 cost = 0.00011104
Epoch: 697 cost = 0.00009078
Epoch: 698 cost = 0.00006859
Epoch: 699 cost = 0.00008439
Epoch: 700 cost = 0.00015013
Epoch: 701 cost = 0.00106485
Epoch: 702 cost = 0.00033894
Epoch: 703 cost = 0.00006475
Epoch: 704 cost = 0.00005897
Epoch: 705 cost = 0.00005801
Epoch: 706 cost = 0.00005701
Epoch: 707 cost = 0.00005634
Epoch: 708 cost = 0.00005592
Epoch: 709 cost = 0.00005528
Epoch: 710 cost = 0.00005452
Epoch: 711 cost = 0.00005393
Epoch: 712 cost = 0.00005345
Epoch: 713 cost = 0.00005312
Epoch: 714 cost = 0.00005298
Epoch: 715 cost = 0.00005258
Epoch: 716 cost = 0.00005243
Epoch: 717 cost = 0.00005276
Epoch: 718 cost = 0.00005345
Epoch: 719 cost = 0.00005226
Epoch: 720 cost = 0.00005112
Epoch: 721 cost = 0.00005121
Epoch: 722 cost = 0.00005420
Epoch: 723 cost = 0.00005234
Epoch: 724 cost = 0.00005044
Epoch: 725 cos

In [78]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [45]:
print(y_test.shape)

(59, 5)


In [69]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.7118644


In [70]:
# 70% train 30% test (NDVIclean)
# Train - 204
# Test - 87
X_train = data[0:204,:]
X_test = data[204:,:]
y_train = label[0:204]
y_test = label[204:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(204, 43)
(87, 43)
(204,)
(87,)


In [71]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(204, 5)
(87, 5)


In [76]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 17
num_batch = 12

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.71196042
Epoch: 2 cost = 0.63074839
Epoch: 3 cost = 0.57540137
Epoch: 4 cost = 0.53940078
Epoch: 5 cost = 0.51618240
Epoch: 6 cost = 0.50071722
Epoch: 7 cost = 0.48979020
Epoch: 8 cost = 0.48159369
Epoch: 9 cost = 0.47519944
Epoch: 10 cost = 0.47013877
Epoch: 11 cost = 0.46612783
Epoch: 12 cost = 0.46292661
Epoch: 13 cost = 0.46030005
Epoch: 14 cost = 0.45803612
Epoch: 15 cost = 0.45597323
Epoch: 16 cost = 0.45400852
Epoch: 17 cost = 0.45208716
Epoch: 18 cost = 0.45018363
Epoch: 19 cost = 0.44828705
Epoch: 20 cost = 0.44639256
Epoch: 21 cost = 0.44449804
Epoch: 22 cost = 0.44260480
Epoch: 23 cost = 0.44071867
Epoch: 24 cost = 0.43885124
Epoch: 25 cost = 0.43701943
Epoch: 26 cost = 0.43524315
Epoch: 27 cost = 0.43354185
Epoch: 28 cost = 0.43192943
Epoch: 29 cost = 0.43040993
Epoch: 30 cost = 0.42897457
Epoch: 31 cost = 0.42760370
Epoch: 32 cost = 0.42627158
Epoch: 33 cost = 0.42495332
Epoch: 34 cost = 0.42363145
Epoch: 35 cost = 0.42229817
Epoch: 36 cost = 0.42095443
E

Epoch: 291 cost = 0.04294822
Epoch: 292 cost = 0.04525703
Epoch: 293 cost = 0.04423145
Epoch: 294 cost = 0.04300449
Epoch: 295 cost = 0.04227629
Epoch: 296 cost = 0.04178108
Epoch: 297 cost = 0.04138870
Epoch: 298 cost = 0.04103285
Epoch: 299 cost = 0.04067845
Epoch: 300 cost = 0.04031040
Epoch: 301 cost = 0.03993106
Epoch: 302 cost = 0.03955832
Epoch: 303 cost = 0.03923103
Epoch: 304 cost = 0.03901943
Epoch: 305 cost = 0.03898792
Epoch: 306 cost = 0.03887966
Epoch: 307 cost = 0.03796412
Epoch: 308 cost = 0.03685976
Epoch: 309 cost = 0.04189276
Epoch: 310 cost = 0.05709846
Epoch: 311 cost = 0.03626121
Epoch: 312 cost = 0.03345713
Epoch: 313 cost = 0.03353760
Epoch: 314 cost = 0.03333351
Epoch: 315 cost = 0.03324634
Epoch: 316 cost = 0.03316187
Epoch: 317 cost = 0.03304547
Epoch: 318 cost = 0.03288663
Epoch: 319 cost = 0.03268504
Epoch: 320 cost = 0.03244544
Epoch: 321 cost = 0.03217370
Epoch: 322 cost = 0.03187470
Epoch: 323 cost = 0.03155078
Epoch: 324 cost = 0.03120027
Epoch: 325 cos

Epoch: 712 cost = 0.00008451
Epoch: 713 cost = 0.00008211
Epoch: 714 cost = 0.00007992
Epoch: 715 cost = 0.00007809
Epoch: 716 cost = 0.00007674
Epoch: 717 cost = 0.00007622
Epoch: 718 cost = 0.00007668
Epoch: 719 cost = 0.00007590
Epoch: 720 cost = 0.00007383
Epoch: 721 cost = 0.00007327
Epoch: 722 cost = 0.00007408
Epoch: 723 cost = 0.00007258
Epoch: 724 cost = 0.00007172
Epoch: 725 cost = 0.00007097
Epoch: 726 cost = 0.00007038
Epoch: 727 cost = 0.00006992
Epoch: 728 cost = 0.00006969
Epoch: 729 cost = 0.00006999
Epoch: 730 cost = 0.00007054
Epoch: 731 cost = 0.00006921
Epoch: 732 cost = 0.00006801
Epoch: 733 cost = 0.00006761
Epoch: 734 cost = 0.00006810
Epoch: 735 cost = 0.00006767
Epoch: 736 cost = 0.00006693
Epoch: 737 cost = 0.00006634
Epoch: 738 cost = 0.00006520
Epoch: 739 cost = 0.00006457
Epoch: 740 cost = 0.00006416
Epoch: 741 cost = 0.00006447
Epoch: 742 cost = 0.00006546
Epoch: 743 cost = 0.00006456
Epoch: 744 cost = 0.00006406
Epoch: 745 cost = 0.00006296
Epoch: 746 cos

In [79]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.7241379
